# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
import re
from joblib import dump, load
from sqlalchemy import create_engine

# import nltk libraries
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# import sklearn libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql("SELECT * FROM Message", engine)
X = df['message']
Y = df.loc[:,'related':'direct_report']

In [3]:
# display head of X
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [4]:
# display head of Y
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    text = text.lower() 
    text = re.sub(r"[^a-zA-Z0-9]", " ", text) 
    words = word_tokenize(text)
    words = [w for w in words if w not in stopwords.words("english")]
    lemmed = [WordNetLemmatizer().lemmatize(w, pos='v') for w in words]
    return lemmed

In [6]:
# test tokenize()
text = X[0]
tokenize(text)

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pass', 'haiti']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
# instantiate Pipeline
pipeline = Pipeline([
    # text tokenizer & tfidf transformer
    ('text_pipeline', Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer())
        ])),
    # estimator
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42)))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
# split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [9]:
# train the model
model = pipeline.fit(X_train, Y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [23]:
# make test predictions 
Y_pred = model.predict(X_test)

In [24]:
# print predictions classification_report()
for index, column in enumerate(Y_test.columns):
    print('Category: **{}**'.format(column))
    print(classification_report(Y_test[column], Y_pred[:,index]))

Category: **related**
             precision    recall  f1-score   support

          0       0.61      0.48      0.53      1524
          1       0.85      0.90      0.87      4990
          2       0.40      0.35      0.37        40

avg / total       0.79      0.80      0.79      6554

Category: **request**
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5432
          1       0.78      0.42      0.55      1122

avg / total       0.87      0.88      0.87      6554

Category: **offer**
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6530
          1       0.00      0.00      0.00        24

avg / total       0.99      1.00      0.99      6554

Category: **aid_related**
             precision    recall  f1-score   support

          0       0.75      0.86      0.80      3867
          1       0.74      0.59      0.66      2687

avg / total       0.75      0.75      0.74      655

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
# get pipeline parameters
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'text_pipeline', 'clf', 'text_pipeline__memory', 'text_pipeline__steps', 'text_pipeline__vect', 'text_pipeline__tfidf', 'text_pipeline__vect__analyzer', 'text_pipeline__vect__binary', 'text_pipeline__vect__decode_error', 'text_pipeline__vect__dtype', 'text_pipeline__vect__encoding', 'text_pipeline__vect__input', 'text_pipeline__vect__lowercase', 'text_pipeline__vect__max_df', 'text_pipeline__vect__max_features', 'text_pipeline__vect__min_df', 'text_pipeline__vect__ngram_range', 'text_pipeline__vect__preprocessor', 'text_pipeline__vect__stop_words', 'text_pipeline__vect__strip_accents', 'text_pipeline__vect__token_pattern', 'text_pipeline__vect__tokenizer', 'text_pipeline__vect__vocabulary', 'text_pipeline__tfidf__norm', 'text_pipeline__tfidf__smooth_idf', 'text_pipeline__tfidf__sublinear_tf', 'text_pipeline__tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimato

In [11]:
# get some of the current param values for the text pipeline
text_params = [
    'text_pipeline__vect__max_df', 
    'text_pipeline__vect__max_features',
    'text_pipeline__tfidf__use_idf'
    ]

for key in text_params:
    print(key + ':', pipeline.get_params()[key])
    print()

text_pipeline__vect__max_df: 1.0

text_pipeline__vect__max_features: None

text_pipeline__tfidf__use_idf: True



In [12]:
# get some of the current param values for the classifier pipeline
clf_params = [
    'clf', 
    'clf__estimator__max_depth',
    'clf__estimator__min_samples_leaf',
    'clf__estimator__n_estimators',
    'clf__estimator'
    ]

for key in clf_params:
    print(key + ':', pipeline.get_params()[key])
    print()

clf: MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
           n_jobs=1)

clf__estimator__max_depth: None

clf__estimator__min_samples_leaf: 1

clf__estimator__n_estimators: 10

clf__estimator: RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)



In [ ]:
# set parameter dictionary for grid search CV
parameters = {
    #'text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
    #'text_pipeline__vect__max_features': (None, 1000),
    'text_pipeline__tfidf__use_idf': (True, False),
    
    #'clf__estimator__min_samples_leaf': [1, 5],
    'clf__estimator__n_estimators': [10, 100]  
    }

# instantiate pipeline with grid search CV
cv = GridSearchCV(pipeline, param_grid=parameters)

In [ ]:
# fit the new pipeline
cv.fit(X_train, Y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# make test predictions 
Y_pred = cv.predict(X_test)

In [ ]:
# print predictions classification_report()
for index, column in enumerate(Y_test.columns):
    print(classification_report(Y_test[column], Y_pred[:,index]))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
dump(cv, 'model-06122019.joblib') 

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.